# CME Gap Analysis

This notebook provides an interactive analysis of CME gaps in Bitcoin price data.


In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path().absolute().parent / 'src'))

from cme_gap_analyzer.data_downloader import download_btc_candles
from cme_gap_analyzer.gap_detector import detect_cme_gaps, find_gap_closures
from cme_gap_analyzer.statistics import calculate_gap_statistics, print_statistics
from cme_gap_analyzer.visualizations import (
    plot_gap_statistics,
    plot_price_action_with_gaps,
    plot_closure_analysis
)

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline


## Download Bitcoin Price Data


In [ ]:
# Download data (adjust dates as needed)
df = download_btc_candles(
    start_date='2022-01-01',
    end_date=None,  # Until now
    interval='1h',
    exchange='binance'
)

print(f"Downloaded {len(df)} candles")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
df.head()


## Detect CME Gaps


In [ ]:
# Detect gaps
gaps_df = detect_cme_gaps(df, local_tz='America/Chicago')

print(f"Detected {len(gaps_df)} CME gaps")
gaps_df.head()


## Find Gap Closures


In [ ]:
# Find closures
gaps_df = find_gap_closures(df, gaps_df)

closed_count = gaps_df['is_closed'].sum()
print(f"Closed gaps: {closed_count}/{len(gaps_df)}")
print(f"Closure rate: {closed_count/len(gaps_df)*100:.2f}%")

# Show closed gaps
closed_gaps = gaps_df[gaps_df['is_closed']]
if not closed_gaps.empty:
    print("\nSample of closed gaps:")
    closed_gaps[['gap_start', 'gap_size', 'days_to_close']].head(10)


## Calculate Statistics


In [ ]:
# Calculate and print statistics
stats = calculate_gap_statistics(gaps_df)
print_statistics(stats)


## Visualizations


In [ ]:
# Plot gap statistics
plot_gap_statistics(gaps_df)


In [ ]:
# Plot price action with gaps
plot_price_action_with_gaps(df, gaps_df)


In [ ]:
# Plot closure analysis
plot_closure_analysis(gaps_df)


## Custom Analysis

You can perform additional custom analysis here.


In [ ]:
# Example: Analyze gaps by year
gaps_df['year'] = pd.to_datetime(gaps_df['gap_start']).dt.year
yearly_summary = gaps_df.groupby('year').agg({
    'gap_size': ['count', 'mean', 'std'],
    'is_closed': 'sum'
})
print(yearly_summary)
